In [1]:
import torch
from supervoice_gpt import SupervoiceGPT, Tokenizer, config

In [2]:
# Model
device = "cpu"
tokenizer = Tokenizer(config, "tokenizer_text.model")
model = SupervoiceGPT(config).to(device)
checkpoint = torch.load(f'./output/big_dataset.pt', map_location=device)
model.load_state_dict(checkpoint['model'])
model.eval()
print('ok')

ok


In [10]:
tokens = model.generate("London is a capital of great britain!", tokenizer, max_new_tokens = 256, top_k = 6, device = device)
print(tokens)

[('l', 9, 108), ('ɐ', 8, 111), ('n', 6, 122), ('d', 2, 123), ('ə', 7, 120), ('n', 6, 118), ('<SIL>', 0, 118), ('ɪ', 6, 116), ('z', 5, 112), ('<SIL>', 0, 107), ('ə', 6, 104), ('<SIL>', 0, 108), ('cʰ', 9, 104), ('æ', 9, 111), ('pʲ', 8, 109), ('ɪ', 3, 111), ('t', 3, 112), ('ə', 5, 111), ('ɫ', 7, 108), ('<SIL>', 0, 110), ('ə', 7, 107), ('v', 5, 104), ('<SIL>', 0, 103), ('ɟ', 6, 103), ('ɹ', 4, 99), ('ej', 11, 105), ('t', 4, 107), ('<SIL>', 0, 107), ('b', 4, 107), ('ɹ', 5, 106), ('ɪ', 5, 106), ('t', 4, 105), ('ə', 3, 100), ('n', 12, 101)]


In [4]:
# input = "Hey, you?"
# ctx_tokens = ['ç', 'ɪ']
# ctx_durations = [8, 10]
# probs, p = model.predict_next(input, ctx_tokens, ctx_durations, tokenizer)
# for i in range(len(probs)):
#     print(p[i] + ": "+ str(probs[i]))
